<a href="https://colab.research.google.com/github/matdekoning/FakeNewsClassifier/blob/master/FakeNewsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fake news classifier**

In [2]:
# First we import the required libraries

import tensorflow as tf
import random
import numpy as np
import pandas as pd
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import io

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K 
K.clear_session()

# Check tensorflow version
if float(tf.__version__[0]) < 2.0:
  print('Updating tensorflow')
  !pip install tensorflow==2.0
else:
  print('Correct version of Tensorflow installed.')

Correct version of Tensorflow installed.


In [3]:
# Retrieve the data

df = pd.read_csv('../data/raw.csv')
df = df.fillna(' ')
df.count()

id        20800
title     20800
author    20800
text      20800
label     20800
dtype: int64

In [4]:
# Tokenize text

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
word_index = tokenizer.word_index
vocab_size=len(word_index)
print(vocab_size)

# Padding data

sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=500, padding='post', truncating='post')

238934


In [5]:
split = 0.25
split_n = int(round(len(padded)*(1-split),0))

train_data = padded[:split_n]
train_labels = df['label'].values[:split_n]
test_data = padded[split_n:]
test_labels = df['label'].values[split_n:]

In [6]:
# Import tensor representations for words
embeddings_index = {};
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;
print(len(coefs))

embeddings_matrix = np.zeros((vocab_size+1, 100));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

100


In [12]:
# Build the architecture of the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, 100, weights=[embeddings_matrix], trainable=False),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         23893500  
_________________________________________________________________
dense_9 (Dense)              (None, None, 32)          3232      
_________________________________________________________________
dense_10 (Dense)             (None, None, 32)          1056      
_________________________________________________________________
dense_11 (Dense)             (None, None, 1)           33        
Total params: 23,897,821
Trainable params: 4,321
Non-trainable params: 23,893,500
_________________________________________________________________


In [19]:
model.input

<tf.Tensor 'embedding_3_input:0' shape=(None, None) dtype=float32>

In [18]:
# history = model.fit(train_data, train_labels, epochs=5, batch_size=100, validation_data=[test_data, test_labels])
history = model.fit(train_data, train_labels, epochs=5, validation_data=[test_data, test_labels])

print("Training Complete")

Train on 15600 samples, validate on 5200 samples
Epoch 1/5
15600/15600 [==============================] - 3s 210us/sample - loss: 0.6433 - accuracy: 0.6316 - val_loss: 0.6398 - val_accuracy: 0.6366
Epoch 2/5
15600/15600 [==============================] - 3s 213us/sample - loss: 0.6433 - accuracy: 0.6317 - val_loss: 0.6386 - val_accuracy: 0.6375
Epoch 3/5
15600/15600 [==============================] - 3s 200us/sample - loss: 0.6434 - accuracy: 0.6316 - val_loss: 0.6386 - val_accuracy: 0.6378
Epoch 4/5
15600/15600 [==============================] - 3s 207us/sample - loss: 0.6430 - accuracy: 0.6319 - val_loss: 0.6395 - val_accuracy: 0.6365
Epoch 5/5
15600/15600 [==============================] - 3s 209us/sample - loss: 0.6433 - accuracy: 0.6319 - val_loss: 0.6407 - val_accuracy: 0.6351
Training Complete


In [ ]:
# Visualize the results:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()